In [1]:
import numpy as np
import pandas as pd
import json

pokemon_data = pd.read_csv('pokemon.csv')
pokemon_data.name = pokemon_data.name.map(lambda p: p.lower())
pokemon_data.columns = pokemon_data.columns.str.replace('^against_', '')

pokemon_evolution_data = pd.read_csv('evolutionData.csv')
pokemon_evolution_data['preevolution_pokedex_number'] = pokemon_evolution_data['preevolution_pokedex_number'].fillna(0.0).astype(int)

def get_strength_and_weakness(pokemon, typeString):
    strong = []
    weak = []
    types = ['dark','dragon', 'electric', 'fairy', 'fight', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']
    for x in types:
        if pokemon[x].values[0] > 1 and x not in typeString:
            weak.append(x)
        elif pokemon[x].values[0] < 1 and x not in typeString:
            strong.append(x)
    return {
        'strong': ', '.join(strong),
        'weak': ', '.join(weak)
    }

def get_classification(pokemon):
    if isinstance(pokemon.type2.values[0], str):
        typeString = pokemon.type1.values[0]  + ', ' + pokemon.type2.values[0] 
    else:
        typeString = pokemon.type1.values[0]
    return typeString

def get_name(id):
    return pokemon_data.loc[pokemon_data.pokedex_number == id].name.values[0]

def get_evolutions(id):
    evolution_data = pokemon_evolution_data.loc[pokemon_evolution_data['preevolution_pokedex_number'] == id]
    if evolution_data.empty:
        return 'undefined'
    evolutions = ''
    for index, row in evolution_data.iterrows():
        evolutions = evolutions + get_name(row['pokedex_number']) + ', '
    return evolutions[:-2]
    

def objectify(pokemon):
    if pokemon.empty:
        return {}
    typeString = get_classification(pokemon)
    effectiveness = get_strength_and_weakness(pokemon, typeString)
    evolutions = get_evolutions(pokemon.pokedex_number.values[0])
    return {
        'name': pokemon.name.values[0] ,
        'type': typeString,
        'classification': pokemon.classfication.values[0],
        'number': str(pokemon.pokedex_number.values[0]),
        'legendary': True if pokemon.is_legendary.values[0] == 1 else False,
        'strong': effectiveness['strong'],
        'weak': effectiveness['weak'],
        'evolutions': evolutions
    }

def get_by_name(name):
    return objectify(pokemon_data.loc[pokemon_data.name == name])



In [3]:
# GET /pokemon
req = json.loads(REQUEST)
args = req['args']
status_code = 200

if 'name' not in args:
    status_code = 404
    print(json.dumps({}))
else:
    pokemon = args['name'][0]
    pokemon = pokemon.lower()
    result = get_by_name(pokemon)
    if not bool(result):
        status_code = 404
    print(json.dumps(result))

In [ ]:
# ResponseInfo GET /pokemon
print(json.dumps({
        'status' : status_code
    })
)